<a href="https://colab.research.google.com/github/ADiko1997/DotNet/blob/master/Covid_Seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from PIL import Image
import PIL
import os
import torchvision.utils
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm.notebook import tqdm
from PIL import ImageFile
from PIL import Image
import random
import dataclasses
from dataclasses import dataclass
from dataclasses import asdict
from sklearn.metrics import f1_score, roc_curve, accuracy_score, auc
from torch.utils.tensorboard import SummaryWriter
import math
import cv2
from IPython.display import Image
import torchvision.models as models
import torchvision.transforms.functional as TF
torch.cuda.empty_cache()

In [ ]:
np.random.seed(0)
random.seed(0)

torch.cuda.manual_seed(0)
torch.manual_seed(0)

In [ ]:
#@title PATHS 
train_img = "/content/train"
train_masks = "/content/train_labels"
test_img = "/content/test"
test_maska = "/content/test_labels"


In [ ]:
#Parameters 
@dataclass
class Config:
 
 
    n_epochs: int = 100  # number of epochs of training
    decay_epoch: int = 5  # epoch from which to start lr decay
 
    img_height: int = 512  # size of image height # default 256x256
    img_width: int = 512  # size of image width
 
    batch_size: int = 4 # size of the batches
    lr: float = 0.01  # adam: learning rate
    b1: float = 0.5  # adam: decay of first order momentum of gradient
 
    channels: int = 3  # number of image channels
    n_cpu: int = 4  # number of cpu threads to use for the dataloaders
    moemntum: int = 0.9
 
 
cfg = Config()

In [ ]:
!wget http://ncov-ai.big.ac.cn/download/ct_lesion_seg.zip


--2020-11-24 17:13:30--  http://ncov-ai.big.ac.cn/download/ct_lesion_seg.zip
Resolving ncov-ai.big.ac.cn (ncov-ai.big.ac.cn)... 124.16.164.43
Connecting to ncov-ai.big.ac.cn (ncov-ai.big.ac.cn)|124.16.164.43|:80... connected.
HTTP request sent, awaiting response... 200 
Length: 884253723 (843M) [application/octet-stream]
Saving to: ‘ct_lesion_seg.zip.1’

ct_lesion_seg.zip.1 100%[===================>] 843.29M  3.64MB/s    in 2m 36s  

2020-11-24 17:16:08 (5.40 MB/s) - ‘ct_lesion_seg.zip.1’ saved [884253723/884253723]



In [ ]:
!unzip ct_lesion_seg.zip >> out.txt

In [ ]:
#@title splitting data
 
import shutil
 
home = os.getcwd()
N_patient = os.listdir(home + "/ct_lesion_seg/mask")
index =0
for i in range(len(N_patient)):
    if N_patient[i] == ".DS_Store":
      # print(i)
      index = i
 
N_patient.pop(index)
 
# for i in range(len(N_patient)):
#     print(N_patient[i],"index ",i)          #creo lista con file dentro$
#del N_patient[0]                                      #rimuovo Ds_store dala lista
random.shuffle(N_patient)                             #dispone in modo casuale i pazienti$
  
    
newpath = home + '/test'
if not os.path.exists(newpath):                     #crea la cartella test
    os.mkdir(newpath)
 
newpath = home + '/test_labels'
if not os.path.exists(newpath):
    os.mkdir(newpath)
    
newpath = home + '/train'
if not os.path.exists(newpath):                     #crea la cartella train
    os.mkdir(newpath)
 
newpath = home + '/train_labels'
if not os.path.exists(newpath):
    os.mkdir(newpath)
   
base_dir = home + '/ct_lesion_seg/mask'                   #definisco due variabili base_d$
base_dir1 = home + '/ct_lesion_seg/image'
 
 
 
for i in range(0,120):
    N_slice=os.listdir(home+"/ct_lesion_seg/mask/"+N_patient[i])              
    for j in range(0,len(N_slice)):
        Name=N_patient[i]+'_'+N_slice[j]
 
        in_name=os.path.join(base_dir,N_patient[i],N_slice[j])
        out_name=os.path.join(base_dir,N_patient[i],Name)             
        os.rename(in_name,out_name)
 
        in_name1=os.path.join(base_dir1,N_patient[i],N_slice[j][:-4]+'.jpg')
        out_name1=os.path.join(base_dir1,N_patient[i],Name[:-4]+'.jpg')           
        os.rename(in_name1,out_name1)
 
        shutil.move(out_name,home+'/train_labels')                                   
        shutil.move(out_name1,home+'/train')
 
for i in range(120,150):
    N_slice=os.listdir(home + "/ct_lesion_seg/mask/" + N_patient[i])             
    for j in range(0,len(N_slice)):
        Name=N_patient[i]+'_'+N_slice[j]
 
        in_name=os.path.join(base_dir,N_patient[i],N_slice[j])
        out_name=os.path.join(base_dir,N_patient[i],Name)
        os.rename(in_name,out_name)
 
        in_name1=os.path.join(base_dir1,N_patient[i],N_slice[j][:-4]+'.jpg')
        out_name1=os.path.join(base_dir1,N_patient[i],Name[:-4]+'.jpg')
        os.rename(in_name1,out_name1)
 
        shutil.move(out_name,home + '/test_labels') 
        shutil.move(out_name1,home + '/test')

In [ ]:
list_os_train_masks = os.listdir(os.path.join(os.getcwd(),"train_labels"))
 
counter_0 =0
counter_1 =0
counter_2 =0
counter_3 =0
 
for i in list_os_train_masks:
    image = cv2.imread(os.path.join(os.getcwd(),"train_labels",i))
    image = np.asarray(image)
    values = np.unique(image, return_counts=True)
 
 
    if len(values[0]) == 1:
        counter_0 += values[1][0]
    
    if len(values[0]) == 2:
        counter_0 += values[1][0]
        counter_1 += values[1][1]
 
    if len(values[0]) == 3:
        counter_0 += values[1][0]
        counter_1 += values[1][1]
        counter_2 += values[1][2]
 
    if len(values[0]) == 4:
        counter_0 += values[1][0]
        counter_1 += values[1][1]
        counter_2 += values[1][2]
        counter_3 += values[1][3]
 
 
num_pixels = 512 * 512 * len(list_os_train_masks) * 3
rate_0 = counter_0/num_pixels
rate_1 = counter_1/num_pixels
rate_2 = counter_2/num_pixels
rate_3 = counter_3/num_pixels
 
 
 
print("Rate 0:",rate_0)
print("Rate 1:",rate_1)
print("Rate 2:",rate_2)
print("Rate 3:",rate_3)
 
weights = [1-rate_0, 1 - rate_1, 1-rate_2, 1-rate_3]

Rate 0: 0.8737735939025879
Rate 1: 0.1146315574645996
Rate 2: 0.007890001932779948
Rate 3: 0.0037048467000325522


In [ ]:
home=os.getcwd()
slices = os.listdir(home+"/train")
first_chann=[]
second_chann=[]
third_chann=[]

std_1=[]
std_2=[]
std_3=[]

for i in range(0,len(slices)):
    
    np_image = np.array(cv2.imread(home+"/train/"+slices[i]))/255

    
    first_chann.append(np_image[:,:,0])
    second_chann.append(np_image[:,:,1])
    third_chann.append(np_image[:,:,2])


first_chann = np.asarray(first_chann)
second_chann = np.asarray(second_chann)
third_chann = np.asarray(third_chann)
   
print("media primo canale: ", np.mean(first_chann), "\n dev st primo canale: ", np.std(first_chann))
print("media secondo canale: ", np.mean(second_chann), "\n dev st seconda canale: ", np.std(second_chann))
print("media terzo canale: ", np.mean(third_chann), "\n dev st terzo canale: ", np.std(third_chann))

media primo canale:  0.527907796273825 
 dev st primo canale:  0.35843899808046975
media secondo canale:  0.527907796273825 
 dev st seconda canale:  0.35843899808046975
media terzo canale:  0.527907796273825 
 dev st terzo canale:  0.35843899808046975


In [ ]:
class COVIDDataSet(Dataset):
    def __init__(self, root,  img_transform=None, mode=None):
        self.root = root
        self.img_transform = img_transform
        
        self.data_dir = root
        self.mask_dir = root+"_labels"
        self.img_list = os.listdir(self.data_dir)
        self.mask_list = os.listdir(self.mask_dir)
        self.mode = mode
 
    def __len__(self):
        return len(self.img_list)

    def backgElim(self, img, mask):

        background = (mask > 0.5)*1
        new_image = np.multiply(img, background)

        return new_image


    def transform(self, image, mask, mode=None):
        

        # Random crop
        # print(image.shape)
        image = torch.from_numpy(image)
        mask = torch.from_numpy(mask)
        # print(mask.shape)
        image = image.permute(2,0,1)
        # mask = image.permute(2,0,1)
        mask = mask.view(1,512,512)
        # print(image.shape)
        # image = torchvision.transforms.ToPILImage(image)
        # mask = torchvision.transforms.ToPILImage(mask)
        if mode == 'train':
          if random.random() > 0.5:
              i, j, h, w = transforms.RandomCrop.get_params(image, output_size=(256,256))
              image = TF.crop(image, i, j, h, w)
              mask = TF.crop(mask, i, j, h, w)


          # Resize
          resize = transforms.Resize(size=(512, 512))
          image = resize(image)
          mask = resize(mask)

          # Random horizontal flipping
          if random.random() > 0.5:
              image = TF.hflip(image)
              mask = TF.hflip(mask)

          # Random vertical flipping
          if random.random() > 0.5:
              image = TF.vflip(image)
              mask = TF.vflip(mask)

        image = image.float()
        image = TF.normalize(image,mean=[0.0501054,0.0501054,0.0501054],std=[0.1438656,0.1438656,0.1438656])

        return image, mask
 
    def __getitem__(self, index):
 
        if self.img_transform == None:
          imgs = cv2.imread(os.path.join(self.data_dir, self.img_list[index]))
          new_index =  self.mask_list.index( self.img_list[index][:-4]+".png")
          labels = cv2.imread(os.path.join(self.mask_dir, self.mask_list[new_index]))
          imgs = self.backgElim(imgs,labels)

          labels = cv2.cvtColor(labels, cv2.COLOR_BGR2GRAY)
          im =os.path.join(self.data_dir, self.img_list[index])
          mask = os.path.join(self.mask_dir, self.mask_list[new_index])
 
        else:
          imgs = cv2.imread(os.path.join(self.data_dir, self.img_list[index]))
          new_index =  self.mask_list.index( self.img_list[index][:-4]+".png")
          labels = cv2.imread(os.path.join(self.mask_dir, self.mask_list[new_index]))
          imgs = self.backgElim(imgs,labels)
          labels = cv2.cvtColor(labels, cv2.COLOR_BGR2GRAY)

          im =os.path.join(self.data_dir, self.img_list[index])
          mask = os.path.join(self.mask_dir, self.mask_list[new_index])
          imgs, labels = self.transform(imgs,labels, self.mode)
 
        return imgs, labels, im, mask


In [ ]:
trainsetCT__ = COVIDDataSet(root=train_img,
                       img_transform= True, mode='train')
 
valsetCT__ = COVIDDataSet(root=test_img,
                     img_transform= True)

In [ ]:
train_loader = DataLoader(trainsetCT__, batch_size=cfg.batch_size, drop_last=False, shuffle=True)
val_loader = DataLoader(valsetCT__, batch_size=cfg.batch_size, drop_last=True, shuffle=True)

In [ ]:
#@title plotting for training
class LinePlotter(object):
    def __init__(self, env_name="main"):
        self.vis = visdom.Visdom(use_incoming_socket=False)
        self.env = env_name
        self.plots = {}

    def plot(self, var_name, split_name, x, y):
        if var_name not in self.plots:
            self.plots[var_name] = self.vis.line(X=np.array([x, x]),
                                    Y=np.array([y, y]), env=self.env, opts=dict(
                                    legend=[split_name],
                                    title=var_name,
                                    xlabel="Iters",
                                    ylabel=var_name
                                    ))
        else:
            self.vis.updateTrace(X=np.array([x, x]), Y=np.array([y, y]), env=self.env,
                                win=self.plots[var_name], name=split_name)

In [ ]:
 #@title visualize images from batch
 
def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
    print("Images:",img.shape)
    img = img.permute(1, 2, 0)
    plt.figure(figsize = (20, 20))
    plt.imshow(img)
    plt.show()
 
def imshow_mask(img):
    img = img * 50     # unnormalize
    img = img.permute(1, 2, 0)
    plt.figure(figsize = (20, 20))
    plt.imshow(img)
    plt.show()
    
dataiter = iter(train_loader)
images, labels_, im, masks_ = dataiter.next()
images_p = images[:4]
labels_p = labels_[:4]
# images = images.permute(0,3,1,2)
# labels_ = labels_.view(10,512,512,1)
# labels_ = labels_.permute(0,3,1,2)
print(labels_p.shape)
# show images
imshow(torchvision.utils.make_grid(images))
# print labels
imshow_mask(torchvision.utils.make_grid(labels_))
print(images[0])

In [ ]:
#Function to make customized layers
def make_layer(block, in_channels, channels, num_blocks, stride=1, dilation=1):
    strides = [stride] + [1]*(num_blocks - 1)

    blocks = []
    for stride in strides:
        blocks.append(block(in_channels=in_channels, channels=channels, stride=stride, dilation=dilation))
        in_channels = block.expansion*channels

    layer = nn.Sequential(*blocks)

    return layer

In [ ]:
#@title Basic Bloc aka traditional Resnet block
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, channels, stride=1, dilation=1):
        super(BasicBlock, self).__init__()

        out_channels = self.expansion*channels

        self.conv1 = nn.Conv2d(in_channels, channels, kernel_size=3, stride=stride, padding=dilation, dilation=dilation, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)

        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=dilation, dilation=dilation, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

        if (stride != 1) or (in_channels != out_channels):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            bn = nn.BatchNorm2d(out_channels)
            self.downsample = nn.Sequential(conv, bn)
        else:
            self.downsample = nn.Sequential()

          
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        out = out + self.downsample(x) #downsample does a sequential exec in case of stride =1

        out = F.relu(out)

        return out

In [ ]:
#@title Bottle neck block -> same as Basic block with an extra Conv + bn layer
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, channels, stride=1, dilation=1):
        super(Bottleneck, self).__init__()

        out_channels = self.expansion*channels

        self.conv1 = nn.Conv2d(in_channels, channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)

        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=stride, padding=dilation, dilation=dilation, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

        self.conv3 = nn.Conv2d(channels, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

        if (stride != 1) or (in_channels != out_channels):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            bn = nn.BatchNorm2d(out_channels)
            self.downsample = nn.Sequential(conv, bn)
        else:
            self.downsample = nn.Sequential()

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.downsample(x)
        out = F.relu(out)
        return out

In [ ]:
#@title Resnets of deeplabV3 -> downsample part

class ResNet_Bottleneck_OS16(nn.Module):
    def __init__(self, num_layers):
        super(ResNet_Bottleneck_OS16, self).__init__()

        if num_layers == 50:
            resnet = models.resnet50(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-3])
            print ("pretrained resnet, 50")
        elif num_layers == 101:
            resnet = models.resnet101(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-3])
            print ("pretrained resnet, 101")
        elif num_layers == 152:
            resnet = models.resnet152(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-3])
        else:
            raise Exception("num_layers must be in {50, 101, 152}!")

        self.layer5 = make_layer(Bottleneck, in_channels=4*256, channels=512, num_blocks=3, stride=1, dilation=2)

    def forward(self, x):
        c4 = self.resnet(x)
        output = self.layer5(c4)

        return output

class ResNet_BasicBlock_OS16(nn.Module):
    def __init__(self, num_layers):
        super(ResNet_BasicBlock_OS16, self).__init__()

        if num_layers == 18:
            resnet = models.resnet18(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-3])
            num_blocks = 2
            print ("pretrained resnet, 18")
        elif num_layers == 34:
            resnet = models.resnet34(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-3])
            num_blocks = 3
            print ("pretrained resnet, 34")
        else:
            raise Exception("num_layers must be in {18, 34}!")

        self.layer5 = make_layer(BasicBlock, in_channels=256, channels=512, num_blocks=num_blocks, stride=1, dilation=2)

    def forward(self, x):
        c4 = self.resnet(x)
        output = self.layer5(c4)

        return output

class ResNet_BasicBlock_OS8(nn.Module):
    def __init__(self, num_layers):
        super(ResNet_BasicBlock_OS8, self).__init__()

        if num_layers == 18:
            resnet = models.resnet18(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-4])
            num_blocks_layer_4 = 2
            num_blocks_layer_5 = 2
            print ("pretrained resnet, 18")
        elif num_layers == 34:
            resnet = models.resnet34(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-4])
            num_blocks_layer_4 = 6
            num_blocks_layer_5 = 3
            print ("pretrained resnet, 34")
        else:
            raise Exception("num_layers must be in {18, 34}!")

        self.layer4 = make_layer(BasicBlock, in_channels=128, channels=256, num_blocks=num_blocks_layer_4, stride=1, dilation=2)

        self.layer5 = make_layer(BasicBlock, in_channels=256, channels=512, num_blocks=num_blocks_layer_5, stride=1, dilation=4)

    def forward(self, x):
        c3 = self.resnet(x)

        output = self.layer4(c3)
        output = self.layer5(output)

        return output

In [ ]:
def ResNet50_OS16():
    return ResNet_Bottleneck_OS16(num_layers=50)

def ResNet50_OS8():
    return ResNet_BasicBlock_OS16(num_layers=18)

def ResNet101_OS16():
    return ResNet_Bottleneck_OS16(num_layers=101)

def ResNet152_OS16():
    return ResNet_Bottleneck_OS16(num_layers=152)

In [ ]:
#@title Upsample
class ASPP(nn.Module):
    def __init__(self, num_classes):
        super(ASPP, self).__init__()

        self.conv_1x1_1 = nn.Conv2d(512, 256, kernel_size=1)
        self.bn_conv_1x1_1 = nn.BatchNorm2d(256)

        self.conv_3x3_1 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=6, dilation=6)
        self.bn_conv_3x3_1 = nn.BatchNorm2d(256)

        self.conv_3x3_2 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=12, dilation=12)
        self.bn_conv_3x3_2 = nn.BatchNorm2d(256)

        self.conv_3x3_3 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=18, dilation=18)
        self.bn_conv_3x3_3 = nn.BatchNorm2d(256)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.conv_1x1_2 = nn.Conv2d(512, 256, kernel_size=1)
        self.bn_conv_1x1_2 = nn.BatchNorm2d(256)

        self.conv_1x1_3 = nn.Conv2d(1280, 256, kernel_size=1)
        self.bn_conv_1x1_3 = nn.BatchNorm2d(256)

        self.conv_1x1_4 = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, feature_map):
        feature_map_h = feature_map.size()[2]
        feature_map_w = feature_map.size()[3]

        out_1x1 = F.relu(self.bn_conv_1x1_1(self.conv_1x1_1(feature_map)))
        out_3x3_1 = F.relu(self.bn_conv_3x3_1(self.conv_3x3_1(feature_map)))
        out_3x3_2 = F.relu(self.bn_conv_3x3_2(self.conv_3x3_2(feature_map)))
        out_3x3_3 = F.relu(self.bn_conv_3x3_3(self.conv_3x3_3(feature_map)))

        out_img = self.avg_pool(feature_map)
        out_img = F.relu(self.bn_conv_1x1_2(self.conv_1x1_2(out_img)))
        out_img = F.upsample(out_img, size=(feature_map_h, feature_map_w), mode="bilinear")

        out = torch.cat([out_1x1, out_3x3_1, out_3x3_2, out_3x3_3, out_img], 1)
        out = F.relu(self.bn_conv_1x1_3(self.conv_1x1_3(out)))
        out = self.conv_1x1_4(out)

        return out

class ASPP_Bottleneck(nn.Module):
    def __init__(self, num_classes):
        super(ASPP_Bottleneck, self).__init__()

        self.conv_1x1_1 = nn.Conv2d(4*512, 256, kernel_size=1)
        self.bn_conv_1x1_1 = nn.BatchNorm2d(256)

        self.conv_3x3_1 = nn.Conv2d(4*512, 256, kernel_size=3, stride=1, padding=6, dilation=6)
        self.bn_conv_3x3_1 = nn.BatchNorm2d(256)

        self.conv_3x3_2 = nn.Conv2d(4*512, 256, kernel_size=3, stride=1, padding=12, dilation=12)
        self.bn_conv_3x3_2 = nn.BatchNorm2d(256)

        self.conv_3x3_3 = nn.Conv2d(4*512, 256, kernel_size=3, stride=1, padding=18, dilation=18)
        self.bn_conv_3x3_3 = nn.BatchNorm2d(256)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.conv_1x1_2 = nn.Conv2d(4*512, 256, kernel_size=1)
        self.bn_conv_1x1_2 = nn.BatchNorm2d(256)

        self.conv_1x1_3 = nn.Conv2d(1280, 256, kernel_size=1)
        self.bn_conv_1x1_3 = nn.BatchNorm2d(256)

        self.conv_1x1_4 = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, feature_map):
        feature_map_h = feature_map.size()[2]
        feature_map_w = feature_map.size()[3]

        out_1x1 = F.relu(self.bn_conv_1x1_1(self.conv_1x1_1(feature_map)))
        out_3x3_1 = F.relu(self.bn_conv_3x3_1(self.conv_3x3_1(feature_map)))
        out_3x3_2 = F.relu(self.bn_conv_3x3_2(self.conv_3x3_2(feature_map)))
        out_3x3_3 = F.relu(self.bn_conv_3x3_3(self.conv_3x3_3(feature_map)))

        out_img = self.avg_pool(feature_map)
        out_img = F.relu(self.bn_conv_1x1_2(self.conv_1x1_2(out_img)))
        out_img = F.upsample(out_img, size=(feature_map_h, feature_map_w), mode="bilinear")

        out = torch.cat([out_1x1, out_3x3_1, out_3x3_2, out_3x3_3, out_img], 1)
        out = F.relu(self.bn_conv_1x1_3(self.conv_1x1_3(out)))
        out = self.conv_1x1_4(out)

        return out

In [ ]:
class DeepLabV3(nn.Module):
    def __init__(self, n_channels = 3, n_classes = 4):
        super(DeepLabV3, self).__init__()
        self.resnet = ResNet50_OS16()
        # self.resnet = ResNet50_OS8()
        self.aspp = ASPP_Bottleneck(num_classes=n_classes)

    def forward(self, x):

        h = x.size()[2]
        w = x.size()[3]

        feature_map = self.resnet(x)

        output = self.aspp(feature_map)

        output = F.upsample(output, size=(h, w), mode="bilinear")

        return output


In [ ]:
if torch.cuda.is_available():
  model = DeepLabV3()
  model.cuda()

In [ ]:
# for name, param in model.named_parameters():
#     if "resnet.resnet" in name or "layer4" in name:
#         param.requires_grad = False
    # print(name)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor(weights))
optimizer = torch.optim.SGD(model.parameters(), lr=cfg.lr, momentum=cfg.moemntum,
                            weight_decay=2e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)
#plotter = LinePlotter()

In [ ]:
#@title training 
# if True:
#     model.train()
#     for epoch in range(cfg.n_epochs):
#         running_loss = 0.0
#         for i, (images, labels_group,im, masks ) in enumerate(train_loader):
#             if torch.cuda.is_available():
#                 images = images.float()
#                 images = images.cuda()
#                 labels_group = labels_group.long()
#                 labels_group = labels_group.cuda()
#             else:
#                 continue
    
#             optimizer.zero_grad()
#             losses = []
#             # images = images.permute(0,3,1,2)
#             # print(images.shape)
#             output = model(images)
#             # print("Labels:",labels_group.shape)
#             labels_group = labels_group.view(cfg.batch_size,512,512)
#             loss = criterion(output.cuda(), labels_group.cuda())
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()
    
#             # lr = lr * (1-(92*epoch+i)/max_iters)**0.9
#             # for parameters in optimizer.param_groups:
#             #     parameters['lr'] = lr
    
#         print("Epoch [%d] Loss: %.4f" % (epoch+1, running_loss/i))
    
#         if (epoch+1) %2 ==0:
#             with torch.no_grad():
#                 val_loss = 0.0
#                 for i, (images, labels_group, im, masks) in enumerate(val_loader):
#                     images = images.float()
#                     images = images.cuda()
#                     labels_group = labels_group.long()
#                     labels_group = labels_group.cuda()
        
#                     images = images.permute(0,3,1,2)
#                     output = model(images)
#                     loss = criterion(output.cuda(), labels_group.cuda())
#                     val_loss+=loss
#                 print("Epoch [%d] Val_Loss: %.4f" % (epoch+1, val_loss/i))
    
#         # ploter.plot("loss", "train", epoch+1, running_loss/i)
#         # running_loss = 0
    
#         # if (epoch+1) % 4 == 0:
#         #     cfg.lr /= 10
#         #     optimizer = torch.optim.SGD(model.parameters())
#         #     torch.save(model.state_dict(), "./pth/fcn-deconv-%d.pth" % (epoch+1))

 
 
# torch.save(model.state_dict(), "./pth/fcn-deconv.pth")

In [ ]:
def load_weight_from_parallel(PATH:str):
  """
  input: PATH -> Path to trained model

  function: nn.Dataparallel models saves the dict_states as modules, meanwhile normal models saves them without modules
            so we want to remove the "module." part from the keys of the dictionary

  output: new dictionary with renamed keys
  """
  parallel_weights = torch.load(PATH)
  p_weights = dict(parallel_weights)
  keys = [k for k in p_weights.keys()]
  new_weights ={}
  for key in keys:
    new_weights[key[7:]] = p_weights[key]

  return new_weights

In [ ]:
weights = load_weight_from_parallel('/content/drive/MyDrive/COV_SEG_MODEL/segnet_norm.pth')
model.load_state_dict(weights)


In [ ]:
def test_analyticaly(model, val_loader):

  model.eval()
  with torch.no_grad():
    val_loss = 0.0
    for i, (images, labels_group, im, masks) in enumerate(val_loader):
      images = images.float()
      images = images.cuda()
      labels_group = labels_group.long()
      labels_group = labels_group.cuda()
      labels_group = labels_group.view(cfg.batch_size,512,512)
      # images = images.permute(0,3,1,2)
      output = model(images)
      loss = criterion(output.cuda(), labels_group.cuda())
      val_loss+=loss
      # print(type(val_loss))
    print(" Val_Loss: %.4f" % (val_loss/i))

In [ ]:
test_analyticaly(model, val_loader)

In [ ]:
dataiter = iter(val_loader)
images, labels_, im, masks_ = dataiter.next()
images_p = images[1]
labels_p = labels_[1]

plt.figure(figsize=(5,5))
images_p = images_p.permute(1,2,0)
plt.imshow(images_p)
print(images_p.shape)
plt.figure(figsize=(5,5))
# labels_p = labels_p.permute(1,2,0)
labels_p = labels_p.view(512,512)
plt.imshow(labels_p)

In [ ]:
model.eval()
images_p = images_p.permute(2,0,1)
images_p = images_p.view(1,3,512,512).float()
output = model(images_p.cuda())

In [ ]:
output_mask = torch.argmax(output.squeeze(), dim=0).detach().cpu().numpy()

In [ ]:
print(np.unique(output_mask))

In [ ]:
label_colors = np.array([(0,0,0),(128,0,0),(0,128,0), (0,0,128)])
r = np.zeros_like(output_mask).astype(np.uint8)
g = np.zeros_like(output_mask).astype(np.uint8)
b = np.zeros_like(output_mask).astype(np.uint8)

for l in range(0, 4):
  idx = output_mask == l
  r[idx] = label_colors[l, 0]
  g[idx] = label_colors[l, 1]
  b[idx] = label_colors[l, 2]

rgb = np.stack([r, g, b], axis=2)
plt.imshow(rgb)
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(labels_p)